## Problem Statement

In the range date of 6 July 2022 - 6 September 2022, finding clusters of Bogor's street characteristics inside the datasets that describe the phenomena or characteristics happened in the street, e.g. traffic jams.

### Data Preparation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Setting
sns.despine(left=True, bottom=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format','{:.4f}'.format)
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set()

<Figure size 640x480 with 0 Axes>

In [2]:
alert = pd.read_csv('..\\dataset\\aggregate_alerts_Kota Bogor.csv')
irregular = pd.read_csv('..\\dataset\\aggregate_median_irregularities_Kota Bogor.csv')
jams = pd.read_csv('..\\dataset\\aggregate_median_jams_Kota Bogor.csv')

In [41]:
alert.head()

,id,time,street,type,total_records,avg_location_long,avg_location_lat
0,5930177,2022-07-06 00:00:00,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,61,106.80051600000002,-6.561997000000004
1,5930178,2022-07-06 00:00:00,unknown,WEATHERHAZARD,61,106.80159300000008,-6.555780000000009
2,5930179,2022-07-06 01:00:00,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,60,106.80051600000002,-6.561997000000004
3,5930180,2022-07-06 01:00:00,unknown,WEATHERHAZARD,60,106.80159300000008,-6.555780000000009
4,5930181,2022-07-06 02:00:00,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,60,106.80051600000002,-6.561997000000004


In [56]:
irregular.head()

,time,street,jam_level,median_length,median_delay_seconds,median_regular_speed,total_records,median_seconds,median_speed,median_jam_level,id,geometry
0,2022-07-06 09:00:00,N9 Jalan Raya Dramaga,4,1878.0000,689.0000,13.7750,18,856.5000,7.3400,4.0000,1492793,"MULTILINESTRING ((106.738284 -6.568848, 106.73..."
1,2022-07-06 09:00:00,N9 Jalan Raya Dramaga,3,1226.0000,339.5000,14.4000,6,447.0000,9.9950,3.0000,1492792,"MULTILINESTRING ((106.73546 -6.565547, 106.735..."
2,2022-07-06 10:00:00,Kapten Yusuf,3,1399.0000,517.0000,18.1500,2,707.0000,7.1100,3.0000,1492794,"LINESTRING (106.785538 -6.61574, 106.785797 -6..."
3,2022-07-06 10:00:00,N9 Underpass KH Soleh Iskandar,4,627.0000,525.0000,23.9000,20,573.5000,3.9300,4.0000,1492797,"MULTILINESTRING ((106.800519 -6.562005, 106.80..."
4,2022-07-06 10:00:00,N9 Jalan Raya Dramaga,4,1420.0000,623.0000,11.7100,19,734.0000,6.5600,4.0000,1492796,"MULTILINESTRING ((106.739572 -6.570715, 106.73..."


In [57]:
jams.head()

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
0,0,2022-07-06 00:00:00.000,32.7100,KOTA BOGOR,Pahlawan,2,784.0000,62.0000,20.3100,4,33469021,2022-07-06,2.0000,"MULTILINESTRING ((106.797269 -6.608899, 106.79..."
1,1,2022-07-06 00:00:00.000,32.7100,KOTA BOGOR,Tol Lingkar Luar Bogor,2,1314.0000,63.0000,37.7100,2,33469023,2022-07-06,2.0000,"LINESTRING (106.787027 -6.559782, 106.787701 -..."
2,2,2022-07-06 00:00:00.000,32.7100,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),5,167.0000,-1.0000,0.0000,61,33469019,2022-07-06,5.0000,"MULTILINESTRING ((106.800516 -6.561997, 106.80..."
3,3,2022-07-06 00:00:00.000,32.7100,KOTA BOGOR,N8 Jalan Raya Sukabumi,3,379.0000,117.5000,8.5950,10,33469020,2022-07-06,3.0000,"MULTILINESTRING ((106.845486 -6.6594, 106.8454..."
4,4,2022-07-06 00:00:00.000,32.7100,KOTA BOGOR,Tol Lingkar Luar Bogor,1,2438.0000,87.0000,42.2700,2,33469022,2022-07-06,1.0000,"LINESTRING (106.787027 -6.559782, 106.787701 -..."


In [7]:
alert.info()

# alert df
    # parse time, parse avg_location to long lat, 
    # null value in street
    # check if date same with time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         9762 non-null   int64  
 1   time                       9762 non-null   object 
 2   kemendagri_kabupaten_kode  9762 non-null   float64
 3   kemendagri_kabupaten_nama  9762 non-null   object 
 4   street                     8827 non-null   object 
 5   type                       9762 non-null   object 
 6   avg_location               9762 non-null   object 
 7   total_records              9762 non-null   int64  
 8   date                       9762 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 686.5+ KB


In [47]:
irregular.info()

# irregular df
    # parse time
    # drop cause_type 100% null
    # check if date same with time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4051 entries, 0 to 4050
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   time                       4051 non-null   object 
 1   kemendagri_kabupaten_kode  4051 non-null   float64
 2   kemendagri_kabupaten_nama  4051 non-null   object 
 3   street                     4051 non-null   object 
 4   jam_level                  4051 non-null   int64  
 5   median_length              4051 non-null   float64
 6   median_delay_seconds       4051 non-null   float64
 7   median_regular_speed       4051 non-null   float64
 8   total_records              4051 non-null   int64  
 9   cause_type                 0 non-null      float64
 10  median_seconds             4051 non-null   float64
 11  median_speed               4051 non-null   float64
 12  date                       4051 non-null   object 
 13  median_jam_level           4051 non-null   float

In [58]:
jams.info()

# irregular df
    # parse time
    # fillna street
    # check if date same with time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102322 entries, 0 to 102321
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 102322 non-null  int64  
 1   time                       102322 non-null  object 
 2   kemendagri_kabupaten_kode  102322 non-null  float64
 3   kemendagri_kabupaten_nama  102322 non-null  object 
 4   street                     100529 non-null  object 
 5   level                      102322 non-null  int64  
 6   median_length              102322 non-null  float64
 7   median_delay               102322 non-null  float64
 8   median_speed_kmh           102322 non-null  float64
 9   total_records              102322 non-null  int64  
 10  id                         102322 non-null  int64  
 11  date                       102322 non-null  object 
 12  median_level               102322 non-null  float64
 13  geometry                   10

In [62]:
# alert.isna().sum() # street column
# alert.duplicated().sum() # no duplication


# irregular.isna().sum() # cause_type 100% null
# irregular.duplicated().sum() # no duplication


# jams.isna().sum() # street column
# jams.duplicated().sum() # no duplication

0

In [19]:
alert['date_time'] = pd.to_datetime(alert.time, format='%Y-%m-%d %H:%M:%S')
alert['date'] = pd.to_datetime(alert.date, format='%Y-%m-%d')
alert['avg_location_long'] = np.vectorize(lambda x: x.replace('[','').replace(']','').split(',')[0])(alert.avg_location)
alert['avg_location_lat'] = np.vectorize(lambda x: x.replace('[','').replace(']','').split(',')[1])(alert.avg_location)


irregular['date_time'] = pd.to_datetime(irregular.time, format='%Y-%m-%d %H:%M:%S')
irregular['date'] = pd.to_datetime(irregular.date, format='%Y-%m-%d')


jams['date_time'] = pd.to_datetime(jams.time, format='%Y-%m-%d %H:%M:%S')
jams['date'] = pd.to_datetime(jams.date, format='%Y-%m-%d')

In [20]:
alert = alert.drop(['time_same_date'] , axis=1) # same information
alert = alert.drop('avg_location', axis=1) # redundant information
alert = alert.drop(['kemendagri_kabupaten_nama', 'kemendagri_kabupaten_kode'], axis=1) # low cardinality


irregular = irregular.drop('cause_type', axis=1)
irregular = irregular.drop(['kemendagri_kabupaten_nama', 'kemendagri_kabupaten_kode'], axis=1) # low cardinality


jams = jams.drop('Unnamed: 0' , axis=1)
jams = jams.drop(['kemendagri_kabupaten_nama', 'kemendagri_kabupaten_kode'], axis=1) # low cardinality

In [21]:
alert.street = alert.street.fillna('unknown')
jams.street = jams.street.fillna('unknown')

In [22]:
alert.sample(5)

,id,time,street,type,total_records,date,avg_location_long,avg_location_lat,date_time
8581,6180861,2022-08-28 16:00:00,Palayu,WEATHERHAZARD,23,2022-08-28,106.81304600000003,-6.581853999999996,2022-08-28 16:00:00
9281,6204781,2022-09-02 17:00:00,Gerbang Tol Sentul Barat,WEATHERHAZARD,24,2022-09-02,106.83561800000003,-6.564517999999996,2022-09-02 17:00:00
2460,6012841,2022-07-23 13:00:00,KH Soleh Iskandar (Jalur Lambat),JAM,34,2022-07-23,106.80308994117648,-6.5613605294117665,2022-07-23 13:00:00
7537,6151581,2022-08-22 20:00:00,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,61,2022-08-22,106.80051600000002,-6.561997000000004,2022-08-22 20:00:00
4643,6074316,2022-08-05 17:00:00,N9 Jalan Raya Dramaga,JAM,90,2022-08-05,106.73839512222223,-6.569284111111107,2022-08-05 17:00:00


In [23]:
irregular.sample(5)

,time,street,jam_level,median_length,median_delay_seconds,median_regular_speed,total_records,median_seconds,median_speed,date,median_jam_level,id,geometry,date_time
2524,2022-08-13 19:00:00.000,Letjen Ibrahim Adjie,4,811.0000,784.0000,18.4400,32,934.0000,3.9450,2022-08-13,4.0000,1554072,"MULTILINESTRING ((106.758743 -6.578083, 106.75...",2022-08-13 19:00:00
3801,2022-09-03 08:00:00.000,Kapten Yusuf,3,1166.0000,391.0000,14.1400,11,545.0000,7.6400,2022-09-03,3.0000,1580396,"MULTILINESTRING ((106.790893 -6.611648, 106.79...",2022-09-03 08:00:00
2756,2022-08-18 07:00:00.000,Mayjen Ishak Djuarsa,4,743.0000,547.0000,21.5600,1,619.0000,4.3200,2022-08-18,4.0000,1559931,"LINESTRING (106.775145 -6.591238, 106.777149 -...",2022-08-18 07:00:00
604,2022-07-16 21:00:00.000,N8 Jalan Raya Bogor,3,880.0000,307.0000,25.6600,1,392.0000,8.0600,2022-07-16,3.0000,1510844,"LINESTRING (106.821189 -6.551776, 106.821824 -...",2022-07-16 21:00:00
1146,2022-07-26 08:00:00.000,Ciomas Raya,4,756.0000,532.5000,16.4950,8,630.5000,4.3100,2022-07-26,4.0000,1527030,"MULTILINESTRING ((106.767461 -6.602699, 106.76...",2022-07-26 08:00:00


In [24]:
jams.sample(5)

,time,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry,date_time
2988,2022-07-07 21:00:00.000,Tol Lingkar Luar Bogor,1,1919.0000,67.0000,43.6900,8,33535189,2022-07-07,1.0000,"MULTILINESTRING ((106.786987 -6.559893, 106.78...",2022-07-07 21:00:00
10444,2022-07-13 10:00:00.000,Atang Sanjaya,1,938.0000,85.0000,15.1300,1,33894206,2022-07-13,1.0000,"LINESTRING (106.752241 -6.545421, 106.751283 -...",2022-07-13 10:00:00
70055,2022-08-16 16:00:00.000,N8 Jalan Raya Sukabumi,3,244.0000,114.0000,5.3700,24,36090038,2022-08-16,3.0000,"MULTILINESTRING ((106.847234 -6.655619, 106.84...",2022-08-16 16:00:00
3045,2022-07-08 00:00:00.000,Tol Jagorawi,1,1178.0000,61.0000,31.8600,3,33598482,2022-07-08,1.0000,"MULTILINESTRING ((106.844761 -6.639406, 106.84...",2022-07-08 00:00:00
970,2022-07-06 16:00:00.000,Brigjen Saptadji Hadi Prawira,3,1112.0000,175.0000,14.4300,8,33469957,2022-07-06,3.0000,"MULTILINESTRING ((106.768857 -6.572383, 106.76...",2022-07-06 16:00:00


### Save all works

In [25]:
alert.to_parquet('..\\dataset\\aggregate_alerts_bogor_preproc.parquet')
irregular.to_parquet('..\\dataset\\aggregate_median_irregularities_bogor_preproc.parquet')
jams.to_parquet('..\\dataset\\aggregate_median_jams_bogor_preproc.parquet')